# Livro para consulta:
- https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html
- https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html
    

# 1. Importando bibliotecas <a name="import"></a>

<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# 2. Carregando o dataframe SINASC <a name="read"></a>
<div style="text-align: right"
     
[Voltar ao índice](#Contents)

In [8]:
# Carregando a Base de Dados
sinasc_raw = pd.read_csv('https://raw.githubusercontent.com/vinisavordelli/exercicios_ebac/main/Ci%C3%AAncia%20de%20Dados/Exercicio_5/SINASC_RO_2019.csv')

## Deixando o formato padrão de floats para 2 casas decimais
pd.options.display.float_format = '{:,.2f}'.format

sinasc_raw.head()


,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,"2,679,477.00",110001,1,19,5.00,8 a 11 anos,NaN,0.00,0.00,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.94,-62.00,338.00,"7,067.02"
1,1,"2,679,477.00",110001,1,29,2.00,8 a 11 anos,"999,992.00",1.00,0.00,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.94,-62.00,338.00,"7,067.02"
2,1,"2,679,477.00",110001,1,37,9.00,8 a 11 anos,"513,205.00",2.00,0.00,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.94,-62.00,338.00,"7,067.02"
3,1,"2,516,500.00",110001,1,30,5.00,12 anos ou mais,"231,205.00",0.00,0.00,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13,-61.85,397.00,"3,958.27"
4,1,"2,516,500.00",110001,1,30,2.00,8 a 11 anos,"999,992.00",1.00,0.00,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.94,-62.00,338.00,"7,067.02"


# Tarefa 1

### 1. Idade media das mães e dos pais por município (coluna munResNome)


In [56]:
sinasc_raw.groupby('munResNome')['IDADEMAE', 'IDADEPAI'].mean().astype(int)

/tmp/ipykernel_387659/4283771480.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  sinasc_raw.groupby('munResNome')['IDADEMAE', 'IDADEPAI'].mean().astype(int)


,IDADEMAE,IDADEPAI
munResNome,,
Alta Floresta D'Oeste,25,29
Alto Alegre dos Parecis,24,29
Alto Paraíso,24,28
Alvorada D'Oeste,25,30
Ariquemes,25,32
Buritis,25,30
Cabixi,26,34
Cacaulândia,25,36
Cacoal,26,30


### 2. Peso médio dos bebes por sexo que nasceram no dia do seu aniversário por faixas de escolaridade mae
Ex: Você, aluna(o), nasceu no dia 10/01, então você precisa filtrar o conjunto de dados nessa data e calcular o peso médio dos bebês de cada sexo por faixa de escolaridade da mãe.

In [75]:
# Checa por dados faltantes nas colunas utilizadas
print(sinasc_raw.shape)
print(sinasc_raw[['DTNASC', 'ESCMAE', 'PESO']].isna().sum())
# Analisa que a unica coluna com dados faltantes é a coluna de escolaridade da mae
print(f"Os valores faltantes de Escolaridade da mãe representam {((sinasc_raw['ESCMAE'].isna().sum() / sinasc_raw.shape[0])*100).round(2)} % da base")
# Dropa os dados faltantes pois são poucos
sinasc_raw['ESCMAE'].dropna()

born_12_08 = sinasc_raw[sinasc_raw['DTNASC'] == '2019-08-12']
## Como o peso é em gramas, utilzaremos a media como valor inteiro.
born_12_08.groupby('ESCMAE')['PESO'].mean().astype(int)


(27028, 69)
DTNASC      0
ESCMAE    312
PESO        0
dtype: int64
Os valores faltantes de Escolaridade da mãe representam 1.15 % da base


ESCMAE
1 a 3 anos         4315
12 anos ou mais    3203
4 a 7 anos         3205
8 a 11 anos        3172
Name: PESO, dtype: int64

### 3. Qual o municipio que nasceu menos bebe em 2019?
    - qual a idade media, maxima, minima das maes nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?

In [37]:
# Pega a cidade com menos nascimento, a variável peso foi escolhida pois não possui dados faltantes
dado = sinasc_raw.groupby('munResNome')['PESO'].count().sort_values(ascending=True).head(1)
print(dado)

# Dropa os municipios ignorados
sinasc_raw = sinasc_raw[sinasc_raw['munResNome'] != 'Município ignorado - RO']


# Reestabelece a variavel dado e cria um uma variavel com o nome da cidade
cidade = sinasc_raw.groupby('munResNome')['PESO'].count().sort_values(ascending=True).head(1)
lowest_city = pd.DataFrame(dado).index[0]

print(f"A cidade com menos nascimentos foi {lowest_city} com {cidade[0]} nascimentos.")

aggregated = sinasc_raw.groupby('munResNome')['IDADEMAE', 'IDADEPAI'].agg([('min', 'min'), ('media', 'mean'), ('max', 'max')]).astype(int)
aggregated[aggregated.index == lowest_city]


munResNome
Castanheiras    32
Name: PESO, dtype: int64
A cidade com menos nascimentos foi Castanheiras com 32 nascimentos.


/tmp/ipykernel_471142/3412314835.py:11: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  aggregated = sinasc_raw.groupby('munResNome')['IDADEMAE', 'IDADEPAI'].agg([('min', 'min'), ('media', 'mean'), ('max', 'max')]).astype(int)


IDADEMAE           IDADEPAI          
                  min media max      min media max
munResNome                                        
Castanheiras       17    27  39       17    30  43

### 4. Qual o municipio que nasceu mais bebe no mês de março?
    - qual a quantidade de filhos vivos media, maxima, minima nesse municipio?
    - qual a idade media, maxima, minima dos pais nesse municipio?



### Analise as respostas encontradas, tire algum insight delas, conte pra gente algo encontrado nos dados. Algo que você julgue relevante e novo pra você.

Exemplo:
- Ah, descobri que a idade mediana das mulheres que deram a luz no ano de 2019 dos municipios x é maior que y.

In [42]:
highest_city = sinasc_raw.groupby('munResNome')['PESO'].count().sort_values(ascending=False).head(1).index[0]

agg = sinasc_raw.groupby('munResNome')['QTDFILVIVO', 'IDADEPAI', 'IDADEMAE'].agg([('min', 'min'), ('media', 'mean'), ('max', 'max')]).astype(int)
agg[agg.index == highest_city]

/tmp/ipykernel_471142/3506090631.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  agg = sinasc_raw.groupby('munResNome')['QTDFILVIVO', 'IDADEPAI', 'IDADEMAE'].agg([('min', 'min'), ('media', 'mean'), ('max', 'max')]).astype(int)


QTDFILVIVO           IDADEPAI           IDADEMAE          
                   min media max      min media max      min media max
munResNome                                                            
Porto Velho          0     1  12       16    32  65       12    26  47

#### Insight:

Apesar de ter sido a cidade que registrou mais nascimentos, podemos notar que a média de filhos vivos é de apenas 1, o que não nos sugere que a cidade tenha uma acima da média. Esse número pode ser fácilmente explicado pois Porto Velho é de longe a maior cidade do estado de Rondônia.

Podemos perceber também que em todos os cortes utilizados, a idade do pai sempre é maior do que a da mãe. Mas para tirar qualquer proveito substancial dessa informação, precisaríamos de um estudo mais aprofundado.